In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def scrape_kryptora_contact():
    url = "https://kryptora.com/contact-us/"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, "html.parser")

    # Extract emails
    emails = set()
    for a in soup.select("a[href^=mailto]"):
        emails.add(a.get_text(strip=True))

    # Extract phone numbers (simple regex)
    phones = set(re.findall(r'\+?\d[\d\s\-]{7,}\d', soup.get_text()))

    # Extract addresses: heuristic approach — find text blocks with keywords
    address_keywords = ["Address", "Office", "Sector", "Pin", "Opp.", "Near"]
    addresses = []
    # Find all divs that contain address-like info
    divs = soup.find_all("div", class_="elementor-widget-container")
    for div in divs:
        text = div.get_text(separator="\n", strip=True)
        if any(keyword in text for keyword in address_keywords):
            addresses.append(text)

    # Deduplicate addresses
    addresses = list(set(addresses))

    # Print results
    print("Emails found:")
    for email in emails:
        print(f"- {email}")

    print("\nPhone numbers found:")
    for phone in phones:
        print(f"- {phone}")

    print("\nAddresses found:")
    for addr in addresses:
        print("-" * 30)
        print(addr)

    # Save to CSV
    data = {
        "Emails": list(emails) + [""]*(max(len(emails), len(phones), len(addresses)) - len(emails)),
        "Phones": list(phones) + [""]*(max(len(emails), len(phones), len(addresses)) - len(phones)),
        "Addresses": addresses + [""]*(max(len(emails), len(phones), len(addresses)) - len(addresses)),
    }

    df = pd.DataFrame(data)
    df.to_csv("kryptora_contact_info.csv", index=False)
    print("\nSaved contact info to kryptora_contact_info.csv")

if __name__ == "__main__":
    scrape_kryptora_contact()


Emails found:
- manish@gmail.com
- manish@kryptora.com

Phone numbers found:
- +91-9149104872
- +91-9528212003
- +91-9711225751

Addresses found:
------------------------------
Email Address
manish@kryptora.com
info@kryptora.com
------------------------------
Head Office : E-48 Sector 3 Noida Pin: 201305
------------------------------
Head Office: E-48 Sector 3 Noida, 201305, Meerut : FN-14 , Star Plaza , Near Baccha Park, 250002
+91-9149104872
info@kryptora.com
------------------------------
FN-14 , Star Plaza , Near Baccha Park, 250002
------------------------------
Postal Address:
------------------------------
Postal Address
B-14 Balaji Complex (Basement), Opp. Subharty University

Vedvyas Puri Phase 1 Meerut
250002

Saved contact info to kryptora_contact_info.csv
